In [1]:
using JuMP
using Flux
using Flux: logitcrossentropy, normalise, onecold, onehotbatch, glorot_uniform, @functor
using Statistics: mean
using Zygote

┌ Warning: CuArrays.jl found cuda, but did not find libcudnn. Some functionality will not be available.
└ @ Flux /home/akshay/.julia/packages/Flux/NpkMm/src/Flux.jl:48


In [8]:
# Initialize hyperparameter arguments
struct Args
    lr::Float64
    repeat::Int
    
    Args() = new(0.5, 110)
end

function get_processed_data(args)
    labels = Flux.Data.Iris.labels()
    features = Flux.Data.Iris.features()

    # Subract mean, divide by std dev for normed mean of 0 and std dev of 1.
    normed_features = normalise(features, dims=2)

    klasses = sort(unique(labels))
    onehot_labels = onehotbatch(labels, klasses)

    # Split into training and test sets, 2/3 for training, 1/3 for test.
    train_indices = [1:3:150 ; 2:3:150]

    X_train = normed_features[:, train_indices]
    y_train = onehot_labels[:, train_indices]

    X_test = normed_features[:, 3:3:150]
    y_test = onehot_labels[:, 3:3:150]

    #repeat the data `args.repeat` times
    train_data = Iterators.repeated((X_train, y_train), args.repeat)
    test_data = (X_test,y_test)

    return train_data, test_data
end

# Accuracy Function
accuracy(x, y, model) = mean(onecold(model(x)) .== onecold(y))

accuracy (generic function with 1 method)

In [20]:
function train()
    args = Args()
    
    #Loading processed data
    train_data, test_data = get_processed_data(args)

    model = Chain(Dense(4, 3), QPLayer(3,3))

#         # defining softmax as an optimization problem
#         _x = cp.Parameter(3)
#         _y = cp.Variable(3)
#         obj = cp.Minimize(-_x.T*_y - cp.sum(cp.entr(_y)))
#         cons = [np.ones(3, dtype=np.float32).T*_y == 1.]
#         prob = cp.Problem(obj, cons)
#         self.softmax = CvxpyLayer(prob, parameters=[_x], variables=[_y])

    # Defining loss function to be used in training
    # For numerical stability, we use here logitcrossentropy
    loss(x, y) = logitcrossentropy(model(x), y)

    # Training
    # Gradient descent optimiser with learning rate `args.lr`
    optimiser = Descent(args.lr)

    println("Starting training.")
    Flux.train!(loss, params(model), train_data, optimiser)

    return model, test_data
end

function test(model, test)
    # Testing model performance on test data 
    X_test, y_test = test
    accuracy_score = accuracy(X_test, y_test, model)

    println("Accuracy: $accuracy_score")
end

model, test_data = train()
test(model, test_data)

Starting training.
Accuracy: 0.94


In [12]:
struct QPLayer
    W::Array{Float64}
    b::Array{Float64}
end

function QPLayer(in::Integer, out::Integer)
  return QPLayer(glorot_uniform(out, in), zeros(out))
end

QPLayer

In [3]:
# turn it into a callable
@functor QPLayer

function (a::QPLayer)(x::AbstractArray)
    W, b = a.W, a.b
    (W*x .+ b)
end

In [17]:
Zygote.@adjoint QPLayer(a, b) = QPLayer(a, b), qp -> throw(Error(qp.W, qp.b))

In [18]:
val, back = Zygote.pullback(QPLayer,2,3)

(QPLayer([-0.69671 -0.737907; 0.463331 0.425518; 0.782222 0.35619], [0.0, 0.0, 0.0]), getfield(Zygote, Symbol("##38#39")){getfield(Main, Symbol("##101#back#20")){getfield(Main, Symbol("##18#19"))}}(getfield(Main, Symbol("##101#back#20")){getfield(Main, Symbol("##18#19"))}(getfield(Main, Symbol("##18#19"))())))

In [19]:
back(QPLayer(3,2))

UndefVarError: UndefVarError: Error not defined

In [48]:
Zygote.refresh()